# Identity Resolution

## Rule-Based Entity Resolution (Three XML Sources)

This notebook implements a reproducible, rule-based entity resolution pipeline for three mapped XML datasets: `Lahman_Mapped.xml`, `Reference_Mapped.xml`, and `Savant_Mapped.xml`.

- **Purpose**: Match the same real-world player-season records across sources, generate high-quality ground truth (GT), and prepare evaluation-ready artifacts.
- **Channel design**:
  - **Main channel (non-ID)**: Blocking on `season_year + name_prefix` (using normalized names), matching with string comparators. Used for training/validation/test.
  - **Audit channel (ID)**: `(player_id, season_year)` only for conservative prelabeling; not used for training/evaluation to avoid leakage.

### Workflow

**0) Setup**
- Unified imports (all modules in Section 0.4)
- Logging configuration and directory setup
- Shared utility functions: `add_name_prefix`, `extract_core_ids`, `merge_candidate_columns`, `create_nonid_keys`

**1) Data loading and quality checks**
- Load three XMLs using PyDI `load_xml`
- Apply name normalization using `name_utils.normalize_name_for_blocking` (handles encoding issues, accents, suffixes)
- Profile missingness, uniqueness, text normalization cardinality, and season-year ranges
- Analyze duplicates (pair-keys and exact-row) and deduplicate Lahman; persist cleaned XML

**2) Candidate generation (blocking)**
- **Non-ID channel**: PyDI `StandardBlocker` on `season_year + normalized_name_prefix` for LR and LS edges
- Uses shared utility function `create_nonid_keys` to ensure consistent blocking key generation
- Blocking keys built from normalized names to handle encoding variations

**3) Similarity scoring**
- PyDI `RuleBasedMatcher` with name comparators (Levenshtein 0.7, Jaccard 0.3) on normalized names
- Uses pre-generated candidate pairs (from Section 2) to avoid duplicate blocking
- Function `score_edge_with_pydi_nonid` accepts candidate DataFrame directly
- Sort candidates by similarity score for stratified sampling

**4) Stratified sampling for GT**
- Score-threshold buckets: low < 0.60, mid ∈ [0.60, 0.99), high ≥ 0.99
- Corner rules: boundary windows and hard pos/neg heuristics; backfill to 500 rows per edge when possible
- Export annotation lists with id1/id2, names, years, birth years, empty `label`, and `source_channel='non_id'`
- Includes manual error cases, accent/suffix variants, and birth year conflicts

**5) Conservative prelabeling (id1/id2-based)**
- Define core identifier by stripping trailing side tag (`|L/|R/|S`) from `id1`/`id2`, so `player_id|season_year` is the core ID
- Uses shared utility function `extract_core_ids`
- **TRUE** when core IDs of `id1` and `id2` are equal
- **FALSE** when core IDs of `id1` and `id2` are different
- All sampled pairs receive either TRUE or FALSE; no labels left empty. Write `samples_{edge}_v1_prelabel_pid.csv`

**6) Ground truth assessment**
- Evaluate labeled GT using `assess_edge` function (returns anomaly cases dictionary)
- Check basic counts, label distribution, similarity-by-label stats, core-ID consistency
- Identify potential inconsistencies: high-similarity FALSE, low-similarity TRUE
- Detailed inspection of anomaly cases (high-similarity FALSE and low-similarity TRUE)

**7) Split ground truth into train/validation/test sets**
- Split by entity groups (player_id-based) to avoid data leakage
- All samples from the same `player_id` stay in the same split
- Use `_load_gt` function for unified GT loading
- Save splits to `gt/splits/` directory for use in subsequent notebooks

### Key Features

- **Name normalization**: Centralized normalization via `name_utils.normalize_name_for_blocking` handles UTF-8 hex escapes, Unicode accents, suffixes (Jr/Sr/II/III), and punctuation
- **Shared utilities**: Reusable functions reduce code duplication and ensure consistency
- **No data leakage**: `player_id` excluded from training features and main blocking; used only for prelabeling and entity-based splitting
- **Unified imports**: All module imports consolidated in Section 0.4 for maintainability

### Outputs

- Cleaned Lahman XML (`data/output/clean/Lahman_Mapped_dedup.xml`)
- Blocked candidate pairs (`cand_lr_nonid`, `cand_ls_nonid`)
- Scored candidate pairs with similarity scores
- Sampled annotation CSVs (`samples_{edge}_v1.csv`)
- Prelabeled CSVs (`samples_{edge}_v1_prelabel_pid.csv`)
- Ground truth splits (`gt/splits/gt_{edge}_{split}.csv`)
- Logs and debug artifacts


## 0. Setup

### 0.0 Import all required modules

Unified imports for the entire notebook.


In [51]:
!pip install -qU uma-pydi

In [52]:
# Unified imports for the entire notebook
import logging
import os
import re
import unicodedata
from collections import defaultdict
from pathlib import Path
from typing import Optional, Sequence

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# PyDI imports
from PyDI.entitymatching import RuleBasedMatcher
from PyDI.entitymatching.blocking import StandardBlocker
from PyDI.entitymatching.comparators import StringComparator
from PyDI.io import load_xml

# Local utility imports
from name_utils import normalize_name_for_blocking

print("✓ All required modules imported")


✓ All required modules imported


### 0.1 DEBUG logging setup

In [53]:
# DEBUG logging setup and file paths for three XML sources
# Enable DEBUG-level logging and write to both console and file
os.makedirs('logs', exist_ok=True)
logging.basicConfig(
    level=logging.DEBUG,
    format='[%(levelname)-5s] %(name)s - %(message)s',
    handlers=[
        logging.FileHandler('logs/pydi.log'),
        logging.StreamHandler()
    ],
    force=True
)
logging.getLogger().debug('Debug logging enabled')


[DEBUG] root - Debug logging enabled


### 0.2 Base directory setup

In [54]:
# Base directory of this project (absolute)
BASE_DIR = Path('/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets')

# Output directory for intermediate results/debug
OUTPUT_DIR = BASE_DIR / 'data' / 'output'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Absolute paths to the three mapped XML files
LAHMAN_FILE    = BASE_DIR / 'Lahman_Mapped.xml'
REFERENCE_FILE = BASE_DIR / 'Reference_Mapped.xml'
SAVANT_FILE    = BASE_DIR / 'Savant_Mapped.xml'

print('BASE_DIR:', BASE_DIR)
print('Output directory:', OUTPUT_DIR)
print('Lahman XML:', LAHMAN_FILE.resolve())
print('Reference XML:', REFERENCE_FILE.resolve())
print('Savant XML:', SAVANT_FILE.resolve())

BASE_DIR: /Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets
Output directory: /Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output
Lahman XML: /Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/Lahman_Mapped.xml
Reference XML: /Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/Reference_Mapped.xml
Savant XML: /Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/Savant_Mapped.xml


### 0.3 Common utility functions

Shared utility functions used throughout the notebook for name processing, ID extraction, and column merging.


In [55]:
def add_name_prefix(df: pd.DataFrame, name_col: str = 'full_name_normalized',
                    out: str = 'name_prefix', n: int = 3, inplace: bool = False) -> pd.DataFrame | None:
    """Build a short name prefix on normalized names for blocking keys.
    
    Args:
        df: DataFrame to process
        name_col: Column name containing the name (default: 'full_name_normalized')
        out: Output column name (default: 'name_prefix')
        n: Number of characters per token (default: 3)
        inplace: If True, modify df in place and return None; otherwise return modified copy
    
    Returns:
        Modified DataFrame if inplace=False, None if inplace=True
    """
    result_df = df if inplace else df.copy()
    
    # Fallback: if normalized column is missing, fall back to raw full_name
    if name_col not in result_df.columns and 'full_name' in result_df.columns:
        name_col = 'full_name'
    
    s = result_df[name_col].astype(str).str.lower().str.strip()
    s = s.str.replace(r'[^\w\s]', ' ', regex=True).str.replace(r'\s+', ' ', regex=True)
    result_df[out] = s.str.split().map(
        lambda toks: ''.join(tok[:n] for tok in toks[:2]) if isinstance(toks, list) else ''
    )
    
    return None if inplace else result_df


def extract_core_ids(df: pd.DataFrame) -> tuple[pd.Series, pd.Series]:
    """Extract core IDs by stripping trailing side tag (|L/|R/|S) from id1 and id2.
    
    Args:
        df: DataFrame with 'id1' and 'id2' columns
    
    Returns:
        Tuple of (core1, core2) Series
    """
    core1 = df['id1'].astype(str).str.replace(r'\|[LRS]$', '', regex=True)
    core2 = df['id2'].astype(str).str.replace(r'\|[LRS]$', '', regex=True)
    return core1, core2


def merge_candidate_columns(correspondences: pd.DataFrame, 
                            df_left: pd.DataFrame, 
                            df_right: pd.DataFrame,
                            left_tag: str, 
                            right_tag: str) -> pd.DataFrame:
    """Merge candidate correspondences with left and right DataFrame columns.
    
    Args:
        correspondences: DataFrame with 'id1' and 'id2' columns (matching _rid values)
        df_left: Left DataFrame
        df_right: Right DataFrame
        left_tag: Tag for left side (e.g., 'L')
        right_tag: Tag for right side (e.g., 'R')
    
    Returns:
        Merged DataFrame with suffixed columns
    """
    l_keep = ['_rid', 'player_id', 'season_year', 'full_name', 'birth_year']
    r_keep = ['_rid', 'player_id', 'season_year', 'full_name', 'birth_year']
    l_use = [c for c in l_keep if c in df_left.columns]
    r_use = [c for c in r_keep if c in df_right.columns]
    
    c = (correspondences
         .merge(df_left[l_use], left_on='id1', right_on='_rid', how='left')
         .merge(df_right[r_use], left_on='id2', right_on='_rid', how='left', 
                suffixes=(f'_{left_tag}', f'_{right_tag}')))
    
    return c


def create_nonid_keys(df: pd.DataFrame, tag: str, inplace: bool = False) -> pd.DataFrame | None:
    """Create _key_nonid and _rid columns for non-ID blocking.
    
    Args:
        df: DataFrame to process
        tag: Side tag (e.g., 'L', 'R', 'S')
        inplace: If True, modify df in place and return None; otherwise return modified copy
    
    Returns:
        Modified DataFrame if inplace=False, None if inplace=True
    
    Creates:
        - _key_nonid: season_year|name_prefix (for blocking)
        - _rid: player_id|season_year|tag (for row identification)
    """
    result_df = df if inplace else df.copy()
    
    # Ensure season_year is numeric
    result_df['season_year'] = pd.to_numeric(result_df['season_year'], errors='coerce')
    
    # Ensure name_prefix exists (create if missing)
    if 'name_prefix' not in result_df.columns:
        add_name_prefix(result_df, 'full_name_normalized', 'name_prefix', n=3, inplace=True)
    
    # Create _key_nonid: season_year|name_prefix
    result_df['_key_nonid'] = result_df['season_year'].astype('Int64').astype(str) + '|' + result_df['name_prefix']
    
    # Create _rid: player_id|season_year|tag
    result_df['_rid'] = (result_df['player_id'].astype(str) 
                         + '|' + result_df['season_year'].astype('Int64').astype(str) 
                         + f'|{tag}')
    
    return None if inplace else result_df


print("✓ Common utility functions defined: add_name_prefix, extract_core_ids, merge_candidate_columns, create_nonid_keys")


✓ Common utility functions defined: add_name_prefix, extract_core_ids, merge_candidate_columns, create_nonid_keys


## 1. Load the XML datasets


In [56]:
# Load mapped XMLs into DataFrames using PyDI
logger = logging.getLogger('data_loading')

# Load
df_lahman    = load_xml(LAHMAN_FILE, name='lahman')
df_reference = load_xml(REFERENCE_FILE, name='reference')
df_savant    = load_xml(SAVANT_FILE, name='savant')

# Optional: annotate sources
for df, src in [(df_lahman, 'lahman'), (df_reference, 'reference'), (df_savant, 'savant')]:
    df.attrs['source'] = src

# Quick sanity prints
print('\nDatasets loaded:')
print('  Lahman   :', len(df_lahman), 'records')
print('  Reference:', len(df_reference), 'records')
print('  Savant   :', len(df_savant), 'records')

logger.info('Loaded datasets - Lahman:%d Reference:%d Savant:%d', len(df_lahman), len(df_reference), len(df_savant))


[INFO ] data_loading - Loaded datasets - Lahman:115450 Reference:15215 Savant:6743



Datasets loaded:
  Lahman   : 115450 records
  Reference: 15215 records
  Savant   : 6743 records


In [57]:
# Apply shared name normalization to all three source DataFrames

for df in (df_lahman, df_reference, df_savant):
    if 'full_name' in df.columns:
        df['full_name_normalized'] = df['full_name'].astype('string').map(normalize_name_for_blocking)
    else:
        df['full_name_normalized'] = ''

print("Applied name_utils.normalize_name_for_blocking to Lahman/Reference/Savant.")

Applied name_utils.normalize_name_for_blocking to Lahman/Reference/Savant.


In [58]:
def inspect_name_normalization(df: pd.DataFrame, name: str, n_samples: int = 20):
    print(f"\n=== {name}: name normalization check ===")
    if 'full_name' not in df.columns or 'full_name_normalized' not in df.columns:
        print("  Missing 'full_name' or 'full_name_normalized'.")
        return

    # Compare raw vs normalized
    raw = df['full_name'].astype(str)
    norm = df['full_name_normalized'].astype(str)

    changed = df[raw != norm]
    print(f"  Total rows      : {len(df):,}")
    print(f"  Changed by norm : {len(changed):,} ({len(changed)/len(df):.2%})")

    # Look for remaining hex escapes or backslashes in normalized names
    suspicious_mask = norm.str.contains(r'\\x[0-9a-fA-F]{2}', regex=True, na=False) | \
                      norm.str.contains(r'\\', regex=True, na=False)
    suspicious = df[suspicious_mask]
    print(f"  Still suspicious: {len(suspicious):,} rows with '\\x..' or '\\'")

    # Show sample of changed names
    print(f"\n  Sample changed names (up to {n_samples}):")
    for _, row in changed.head(n_samples).iterrows():
        print(f"    '{row['full_name']}'  ->  '{row['full_name_normalized']}'")

    # Show sample of suspicious normalized names
    if len(suspicious) > 0:
        print(f"\n  Sample suspicious normalized names (up to {n_samples}):")
        for _, row in suspicious.head(n_samples).iterrows():
            print(f"    '{row['full_name']}'  ->  '{row['full_name_normalized']}'")
    else:
        print("\n  No suspicious patterns found in normalized names.")

inspect_name_normalization(df_lahman, "Lahman")
inspect_name_normalization(df_reference, "Reference")
inspect_name_normalization(df_savant, "Savant")


=== Lahman: name normalization check ===
  Total rows      : 115,450
  Changed by norm : 1 (0.00%)
  Still suspicious: 0 rows with '\x..' or '\'

  Sample changed names (up to 20):
    'c v matteson'  ->  'c matteson'

  No suspicious patterns found in normalized names.

=== Reference: name normalization check ===
  Total rows      : 15,215
  Changed by norm : 1,939 (12.74%)
  Still suspicious: 0 rows with '\x..' or '\'

  Sample changed names (up to 20):
    'eli\xc3\xa9zer alfonzo'  ->  'eliezer alfonzo'
    'mois\xc3\xa9s alou'  ->  'moises alou'
    'alfredo am\xc3\xa9zaga'  ->  'alfredo amezaga'
    'alberto \xc3\x81rias'  ->  'alberto arias'
    'joaqu\xc3\xadn arias'  ->  'joaquin arias'
    'jos\xc3\xa9 bautista'  ->  'jose bautista'
    '\xc3\x89rik b\xc3\xa9dard'  ->  'erik bedard'
    'carlos beltr\xc3\xa1n'  ->  'carlos beltran'
    'adrian beltr\xc3\xa9'  ->  'adrian beltre'
    '\xc3\x81ngel berroa'  ->  'angel berroa'
    'gr\xc3\xa9gor blanco'  ->  'gregor blanco'
    

## 2. Data quality checks and usability assessment


In [59]:
# Lightweight profiling of key data quality aspects
logger = logging.getLogger('quality_checks')

def safe_parse_dates(series: pd.Series) -> pd.Series:
    try:
        return pd.to_datetime(series, errors='coerce')
    except Exception:
        return pd.Series([pd.NaT] * len(series))

def quick_profile(
    df: pd.DataFrame,
    name: str,
    candidate_id_cols: Sequence[str] = ("player_id",),
    text_cols: Sequence[str] = ("full_name",),
    date_cols: Sequence[str] = ("season_year",),
    combo_key_cols: Sequence[str] = ("player_id", "season_year"),
) -> None:
    print(f"\n== {name} ==")
    print('rows x cols:', df.shape)
    # Show columns
    print('columns:', list(df.columns)[:20], '...')

    # Missing rate (top 8)
    miss = df.isna().mean().sort_values(ascending=False)
    if len(miss) > 0:
        print('missing ratio (top):')
        print((miss.head(8) * 100).round(2).astype(str) + '%')

    # Candidate key uniqueness (single-column)
    single_key_found = False
    for k in candidate_id_cols:
        if k in df.columns:
            single_key_found = True
            null_ratio = df[k].isna().mean()
            dup_ratio = 1.0 - (df[k].nunique(dropna=True) / max(len(df), 1))
            print(f"key[{k}] null%={null_ratio:.2%} dup%={dup_ratio:.2%}")
            break
    if not single_key_found:
        print('key: no obvious single-column id found in', candidate_id_cols)

    # Combination key uniqueness (e.g., player_id + season_year)
    if combo_key_cols and all(c in df.columns for c in combo_key_cols):
        combo_null = df[list(combo_key_cols)].isna().any(axis=1).mean()
        nunique_combo = df[list(combo_key_cols)].astype(str).agg('|'.join, axis=1).nunique(dropna=True)
        combo_dup = 1.0 - (nunique_combo / max(len(df), 1))
        print(f"combo_key{tuple(combo_key_cols)} null_row%={combo_null:.2%} dup%={combo_dup:.2%}")
    elif combo_key_cols:
        print(f"combo_key missing cols: {combo_key_cols}")
    else:
        print('combo_key check: skipped')

    # Text columns uniqueness before/after normalization
    for c in text_cols:
        if c in df.columns:
            raw_u = df[c].nunique(dropna=True)
            norm = df[c].astype(str).str.strip().str.lower()
            norm_u = norm.nunique(dropna=True)
            print(f"text[{c}] unique(raw->{raw_u}, norm->{norm_u})")

    # Date/year columns: parse/inspect year distribution
    for c in date_cols:
        if c in df.columns:
            # If it's numeric year, compute distribution directly
            if pd.api.types.is_numeric_dtype(df[c]):
                yrs = pd.to_numeric(df[c], errors='coerce').dropna()
                year_range: Optional[tuple] = (int(yrs.min()), int(yrs.max())) if len(yrs) else None
                print(f"year[{c}] range={year_range}")
                if len(yrs):
                    vc = yrs.value_counts().sort_index()
                    head = vc.head(10)
                    tail = vc.tail(3) if len(vc) > 10 else pd.Series(dtype='int64')
                    print('year distribution (head):')
                    print(head.to_string())
                    if len(tail):
                        print('...')
                        print(tail.to_string())
            else:
                d = safe_parse_dates(df[c])
                fail = d.isna().mean()
                yrs = d.dt.year.dropna()
                year_range: Optional[tuple] = (int(yrs.min()), int(yrs.max())) if len(yrs) else None
                print(f"date[{c}] parse_fail%={fail:.2%} year_range={year_range}")

# Run checks for three datasets with updated defaults (single key only)
quick_profile(df_lahman, 'Lahman', candidate_id_cols=("player_id",), text_cols=("full_name",), date_cols=("season_year",), combo_key_cols=())
quick_profile(df_reference, 'Reference', candidate_id_cols=("player_id",), text_cols=("full_name",), date_cols=("season_year",), combo_key_cols=())
quick_profile(df_savant, 'Savant', candidate_id_cols=("player_id",), text_cols=("full_name",), date_cols=("season_year",), combo_key_cols=())

logger.info('Quality checks completed')



== Lahman ==
rows x cols: (115450, 24)
columns: ['player_id', 'full_name', 'birth_year', 'season_year', 'league', 'team', 'games', 'at_bats', 'runs', 'hits', 'doubles', 'triples', 'home_runs', 'runs_batted_in', 'walks', 'intentional_walks', 'strikeouts', 'hit_by_pitch', 'sac_hits', 'sac_flies'] ...
missing ratio (top):
intentional_walks          31.75%
sac_flies                  31.27%
ground_into_double_play    22.04%
caught_stealing            20.39%
sac_hits                    5.26%
hit_by_pitch                2.44%
stolen_bases                2.05%
strikeouts                  1.82%
dtype: object
key[player_id] null%=0.05% dup%=7.71%
combo_key check: skipped
text[full_name] unique(raw->20325, norm->20325)
date[season_year] parse_fail%=0.00% year_range=(1871, 2024)

== Reference ==
rows x cols: (15215, 26)
columns: ['player_id', 'full_name', 'birth_year', 'season_year', 'level', 'team', 'games', 'plate_appearances', 'at_bats', 'runs', 'hits', 'doubles', 'triples', 'home_runs', 'runs

[INFO ] quality_checks - Quality checks completed


### 2.a Duplicate check using composite key: player_id + full_name


In [60]:
def dup_report(df: pd.DataFrame, name: str, id_col='player_id', name_col='full_name', show_examples=5):
    print(f"\n== {name}: duplicate report for ({id_col}, {name_col}) ==")
    if id_col not in df.columns or name_col not in df.columns:
        print('missing required columns')
        return
    key_df = df[[id_col, name_col]].copy()
    key_df[name_col] = key_df[name_col].astype(str).str.strip().str.lower()

    n = len(key_df)
    nunique_pairs = key_df.astype(str).agg('|'.join, axis=1).nunique(dropna=True)
    dup_ratio = 1.0 - (nunique_pairs / max(n, 1))

    # Row-level duplicate mask
    mask_dup = key_df.duplicated(keep=False)
    dup_rows = mask_dup.sum()
    dup_row_ratio = dup_rows / max(n, 1)

    print(f'total rows={n}  unique_pairs={nunique_pairs}  pair_dup_ratio={dup_ratio:.2%}  dup_row_ratio={dup_row_ratio:.2%}')

    # Show top offending keys
    vc = key_df.astype(str).agg('|'.join, axis=1).value_counts()
    offenders = vc[vc > 1].head(show_examples)
    if len(offenders) == 0:
        print('no duplicate pairs found')
    else:
        print('top duplicate keys (pair,count):')
        for k, c in offenders.items():
            pid, nm = k.split('|', 1)
            print(f'  ({pid}, {nm}) -> {c}')

# Run for three datasets
dup_report(df_lahman, 'Lahman')
dup_report(df_reference, 'Reference')
dup_report(df_savant, 'Savant')



== Lahman: duplicate report for (player_id, full_name) ==
total rows=115450  unique_pairs=106583  pair_dup_ratio=7.68%  dup_row_ratio=14.89%
top duplicate keys (pair,count):
  (nan, jack taylor) -> 11
  (nan, darby o brien) -> 6
  (1135201892, tom dowse) -> 5
  (1162691904, frank huelsman) -> 5
  (6575082024, mike baumann) -> 5

== Reference: duplicate report for (player_id, full_name) ==
total rows=15215  unique_pairs=15215  pair_dup_ratio=0.00%  dup_row_ratio=0.00%
no duplicate pairs found

== Savant: duplicate report for (player_id, full_name) ==
total rows=6743  unique_pairs=6743  pair_dup_ratio=0.00%  dup_row_ratio=0.00%
no duplicate pairs found


### 2.b Duplicate check on Lahman using composite key: player_id + team +games


In [61]:
# Duplicate report for Lahman on (player_id, team, games)

req = ['player_id', 'team', 'games']
missing = [c for c in req if c not in df_lahman.columns]
if missing:
    print('Lahman is missing required columns:', missing)
else:
    key_df = df_lahman[req].copy()
    # normalize
    key_df['team'] = key_df['team'].astype(str).str.strip().str.lower()
    key_df['games'] = pd.to_numeric(key_df['games'], errors='coerce')

    n = len(key_df)
    pair_keys = key_df.astype(str).agg('|'.join, axis=1)
    nunique_pairs = pair_keys.nunique(dropna=True)
    pair_dup_ratio = 1.0 - (nunique_pairs / max(n, 1))

    dup_mask = pair_keys.duplicated(keep=False)
    dup_row_ratio = dup_mask.mean()

    print(f"Lahman (player_id, team, games): total={n} unique_pairs={nunique_pairs} "
          f"pair_dup_ratio={pair_dup_ratio:.2%} dup_row_ratio={dup_row_ratio:.2%}")

    offenders = pair_keys.value_counts()
    offenders = offenders[offenders > 1].head(10)
    if len(offenders) == 0:
        print('no duplicate (player_id, team, games) triples found')
    else:
        print('top duplicate (player_id, team, games) triples:')
        for k, c in offenders.items():
            pid, team, games = k.split('|', 2)
            print(f'  ({pid}, {team}, {games}) -> {c}')


Lahman (player_id, team, games): total=115450 unique_pairs=115439 pair_dup_ratio=0.01% dup_row_ratio=0.02%
top duplicate (player_id, team, games) triples:
  (1127931890, ny1, 2) -> 2
  (1122561914, brf, 3) -> 2
  (1102121972, oak, 10) -> 2
  (1196181925, sln, 1) -> 2
  (1140751971, oak, 2) -> 2
  (nan, br2, 2) -> 2
  (2795702004, tex, 8) -> 2
  (1146061888, ws8, 1) -> 2
  (1161002000, ari, 4) -> 2
  (6683382023, lan, 1) -> 2


### 2.c Full-row duplicate check (all columns)


In [62]:
print('\n== Exact duplicate rows (ALL columns) - Lahman ==')

n_total = len(df_lahman)
# Mask of rows that have an identical predecessor across ALL columns
mask_exact_dups_any = df_lahman.duplicated(keep=False)
mask_exact_dups_first = df_lahman.duplicated(keep='first')
num_rows_in_duplicated_groups = int(mask_exact_dups_any.sum())
num_exact_dup_rows_excluding_first = int(mask_exact_dups_first.sum())

print(f'total rows = {n_total}')
print(f'rows in duplicated groups (keep=False) = {num_rows_in_duplicated_groups}')
print(f'exact duplicate rows (excluding first occurrences) = {num_exact_dup_rows_excluding_first}')

if num_rows_in_duplicated_groups > 0:
    # Group by all columns to find groups with count > 1
    grp_sizes = (df_lahman
                 .groupby(list(df_lahman.columns), dropna=False)
                 .size()
                 .reset_index(name='count')
                 .sort_values('count', ascending=False))
    dup_groups = grp_sizes[grp_sizes['count'] > 1]
    print(f'number of unique duplicate groups = {len(dup_groups)}')
    print('top duplicate groups (show up to 5):')
    display(dup_groups.head(5))
else:
    print('No exact duplicate rows found.')



== Exact duplicate rows (ALL columns) - Lahman ==
total rows = 115450
rows in duplicated groups (keep=False) = 6
exact duplicate rows (excluding first occurrences) = 3
number of unique duplicate groups = 3
top duplicate groups (show up to 5):


,player_id,full_name,birth_year,season_year,league,team,games,at_bats,runs,hits,...,intentional_walks,strikeouts,hit_by_pitch,sac_hits,sac_flies,ground_into_double_play,stolen_bases,caught_stealing,full_name_normalized,count
34191,1161002000,darren holmes,1966,2000,NL,ARI,4,0,0,0,...,0,0,0,0,0,0,0,0,darren holmes,2
113618,6683382023,tyson miller,1995,2023,NL,LAN,1,0,0,0,...,0,0,0,0,0,0,0,0,tyson miller,2
15692,1127931890,sam crane,1854,1890,NL,NY1,2,6,0,0,...,NaN,0,0,NaN,NaN,NaN,1,NaN,sam crane,2


## 3. Deduplicate Lahman


### 3.1 Exact-row Business-key deduplication

In [63]:
log = logging.getLogger('dedup')

# --- Step 1: exact-row dedup ---
before = len(df_lahman)
df_lahman = df_lahman.drop_duplicates()
after = len(df_lahman)
print(f'[Exact dedup] Lahman: {before} -> {after} (removed {before-after})')
log.info('Exact dedup finished: %d -> %d', before, after)

# Helpers

def report_dup_ratio(df: pd.DataFrame, key_cols):
    keys = df[key_cols].astype(str).agg('|'.join, axis=1)
    nunique = keys.nunique(dropna=True)
    dup_ratio = 1.0 - (nunique / max(len(df), 1))
    dup_rows = keys.duplicated(keep=False).mean()
    print(f'  key={tuple(key_cols)} dup_ratio={dup_ratio:.2%} dup_row_ratio={dup_rows:.2%}')
    return dup_ratio, dup_rows


def dedup_by_business_key(df: pd.DataFrame, key_cols, prefer_desc_cols=None):
    """Keep one row per key using a deterministic preference:
    - Fewer NaNs first
    - Then by prefer_desc_cols (e.g., games, plate_appearances) descending
    - Stable tie-breaker by index ascending
    """
    if prefer_desc_cols is None:
        prefer_desc_cols = []
    work = df.copy()
    work['_na_cnt'] = work.isna().sum(axis=1)
    sort_cols = list(key_cols) + ['_na_cnt'] + prefer_desc_cols
    ascending = [True]*len(key_cols) + [True] + [False]*len(prefer_desc_cols)
    work = work.sort_values(sort_cols, ascending=ascending, kind='mergesort')
    result = work.drop_duplicates(subset=key_cols, keep='first').drop(columns=['_na_cnt'])
    return result

# --- Step 2: Primary key dedup on player_id only ---
key = [c for c in ['player_id'] if c in df_lahman.columns]
print('\n[Primary key check] Before dedup:')
report_dup_ratio(df_lahman, key)

prefer_cols = [c for c in ['games','plate_appearances','at_bats','hits','runs'] if c in df_lahman.columns]

df_lahman = dedup_by_business_key(df_lahman, key, prefer_desc_cols=prefer_cols)
print('[Primary key dedup] Applied dedup on', tuple(key))
report_dup_ratio(df_lahman, key)
print('Final Lahman rows:', len(df_lahman))


[INFO ] dedup - Exact dedup finished: 115450 -> 115447


[Exact dedup] Lahman: 115450 -> 115447 (removed 3)

[Primary key check] Before dedup:
  key=('player_id',) dup_ratio=7.70% dup_row_ratio=14.91%
[Primary key dedup] Applied dedup on ('player_id',)
  key=('player_id',) dup_ratio=0.00% dup_row_ratio=0.00%
Final Lahman rows: 106553


### 3.2 Save deduplicated Lahman to XML


In [64]:
# Write df_lahman (deduplicated) to XML

save_dir = OUTPUT_DIR / 'clean'
save_dir.mkdir(parents=True, exist_ok=True)

xml_path = save_dir / 'Lahman_Mapped_dedup.xml'

# pandas >=1.3 supports DataFrame.to_xml
# Use generic structure: <dataset><record>...</record>...</dataset>
df_lahman.to_xml(
    xml_path,
    index=False,
    root_name='dataset',
    row_name='record',
    encoding='utf-8'
)

print('Saved deduplicated Lahman XML to:', xml_path)
print('Rows written:', len(df_lahman))


Saved deduplicated Lahman XML to: /Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/clean/Lahman_Mapped_dedup.xml
Rows written: 106553


In [65]:
# Quick size check for the three source tables
# Step 5.0: Normalize inputs (adds *_norm columns, keeps originals)

L, R, S = df_lahman.copy(), df_reference.copy(), df_savant.copy()

print('Normalized dataframes ready:', len(L), len(R), len(S))

Normalized dataframes ready: 106553 15215 6743


## 4. Build candidate pairs

### 4.1 Prepare normalized names and blocking keys

### 4.2 Build candidates on (season_year + name_prefix)

This section builds the main candidate channel for LR and LS by blocking on `season_year + name_prefix` using PyDI `StandardBlocker`. 
It does not rely on `player_id`, so the resulting candidate pairs can be used directly for training, validation, and evaluation of 
blocking and matching methods.

In [66]:
# NON-ID candidates: StandardBlocker on (season_year + normalized name_prefix)
# Note: Uses shared utility functions from Section 0.3

def build_nonid_candidates(left: pd.DataFrame, right: pd.DataFrame, lt: str, rt: str) -> pd.DataFrame:
    """Build non-ID candidate pairs using StandardBlocker on season_year + name_prefix.
    
    Uses shared utility functions: create_nonid_keys, merge_candidate_columns
    """
    Lx = left.copy(); Rx = right.copy()
    
    # Create non-ID keys using shared function (creates _key_nonid and _rid)
    create_nonid_keys(Lx, lt, inplace=True)
    create_nonid_keys(Rx, rt, inplace=True)

    blocker = StandardBlocker(Lx, Rx, on=['_key_nonid'], id_column='_rid', output_dir=OUTPUT_DIR / 'blocking-evaluation')
    pairs = blocker.materialize()

    # attach context using shared merge function
    c = merge_candidate_columns(pairs, Lx, Rx, lt, rt)

    # attach normalized names for inspection (do not overwrite raw full_name)
    if 'full_name_normalized' in Lx.columns:
        left_norm = Lx.set_index('_rid')['full_name_normalized']
        c['full_name_norm_L'] = c['id1'].map(left_norm)
    if 'full_name_normalized' in Rx.columns:
        right_norm = Rx.set_index('_rid')['full_name_normalized']
        c['full_name_norm_R'] = c['id2'].map(right_norm)

    # standardized columns (merge creates suffixed columns already)
    if f'season_year_{lt}' not in c.columns and 'season_year' in c.columns:
        c[f'season_year_{lt}'] = c['season_year']
    if f'season_year_{rt}' not in c.columns and 'season_year' in c.columns:
        c[f'season_year_{rt}'] = c['season_year']
    c['block_type'] = 'non_id_eq'

    # keep compact
    keep = [
        'id1', 'id2',
        f'player_id_{lt}' if f'player_id_{lt}' in c.columns else None,
        f'player_id_{rt}' if f'player_id_{rt}' in c.columns else None,
        f'season_year_{lt}', f'season_year_{rt}',
        f'full_name_{lt}' if f'full_name_{lt}' in c.columns else None,
        f'full_name_{rt}' if f'full_name_{rt}' in c.columns else None,
        f'birth_year_{lt}' if f'birth_year_{lt}' in c.columns else None,
        f'birth_year_{rt}' if f'birth_year_{rt}' in c.columns else None,
        'full_name_norm_L' if 'full_name_norm_L' in c.columns else None,
        'full_name_norm_R' if 'full_name_norm_R' in c.columns else None,
        'block_type'
    ]
    keep = [k for k in keep if k and k in c.columns]
    return c[keep]


# Build NON-ID candidates for three edges
cand_lr_nonid = build_nonid_candidates(L, R, 'L', 'R')
# cand_rs_nonid = build_nonid_candidates(R, S, 'R', 'S')  # RS skipped for now
cand_ls_nonid = build_nonid_candidates(L, S, 'L', 'S')

print('Built NON-ID candidates for LR/RS/LS (season_year + normalized name_prefix)')

[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - Creating blocking key values for dataset1: 106553 records
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - Creating blocking key values for dataset2: 15215 records
[INFO ] PyDI.entitymatching.blocking.standard.StandardBlocker - created 105016 blocking keys for first dataset
[INFO ] PyDI.entitymatching.blocking.standard.StandardBlocker - created 15018 blocking keys for second dataset
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - Joining blocking key values: 105016 x 15018 blocks
[INFO ] PyDI.entitymatching.blocking.standard.StandardBlocker - created 14961 blocks from blocking keys
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - Block size distribution:
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - Size Frequency
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - 14603       1
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - 171   

Built NON-ID candidates for LR/RS/LS (season_year + normalized name_prefix)


### 4.3 Inspect candidate quality

In [67]:
cand_lr_nonid

,id1,id2,player_id_L,player_id_R,season_year_L,season_year_R,full_name_L,full_name_R,birth_year_L,birth_year_R,full_name_norm_L,full_name_norm_R,block_type
0,1100292008|2008|L,1100292008|2008|R,1100292008,1100292008,2008,2008,bobby abreu,bobby abreu,1974,1974,bobby abreu,bobby abreu,non_id_eq
1,1100292009|2009|L,1100292009|2009|R,1100292009,1100292009,2009,2009,bobby abreu,bobby abreu,1974,1974,bobby abreu,bobby abreu,non_id_eq
2,1100292010|2010|L,1100292010|2010|R,1100292010,1100292010,2010,2010,bobby abreu,bobby abreu,1974,1974,bobby abreu,bobby abreu,non_id_eq
3,1100292011|2011|L,1100292011|2011|R,1100292011,1100292011,2011,2011,bobby abreu,bobby abreu,1974,1974,bobby abreu,bobby abreu,non_id_eq
4,1100292012|2012|L,1100292012|2012|R,1100292012,1100292012,2012,2012,bobby abreu,bobby abreu,1974,1974,bobby abreu,bobby abreu,non_id_eq
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15874,8053732024|2024|L,8053732024|2024|R,8053732024,8053732024,2024,2024,nacho alvarez,nacho alvarez jr,2003,2003,nacho alvarez,nacho alvarez,non_id_eq
15875,8057792024|2024|L,8057792024|2024|R,8057792024,8057792024,2024,2024,jacob wilson,jacob wilson,2002,2002,jacob wilson,jacob wilson,non_id_eq
15876,8077992023|2023|L,8077992023|2023|R,8077992023,8077992023,2023,2023,masataka yoshida,masataka yoshida,1993,1994,masataka yoshida,masataka yoshida,non_id_eq
15877,8077992024|2024|L,8077992024|2024|R,8077992024,8077992024,2024,2024,masataka yoshida,masataka yoshida,1993,1994,masataka yoshida,masataka yoshida,non_id_eq


In [68]:
def inspect_garbled_names(df: pd.DataFrame,
                          name: str,
                          cols=('full_name_norm_L', 'full_name_norm_R'),
                          n_samples: int = 20) -> None:
    """Check for garbled names (hex escapes, double backslashes, non-ASCII)."""
    print(f"\n=== {name}: garbled name check ===")

    # Start with an all-False boolean mask over rows
    bad_mask = pd.Series(False, index=df.index)

    for col in cols:
        if col not in df.columns:
            continue
        s = df[col].astype(str)

        # Patterns for typical encoding artifacts (\xHH, over-escaped backslashes)
        hex_mask = s.str.contains(r'\\x[0-9a-fA-F]{2}', regex=True, na=False)
        backslash_mask = s.str.contains(r'\\\\', regex=True, na=False)

        # Non-ASCII characters (after our pipeline these are suspicious)
        non_ascii_mask = s.str.contains(r'[^\x00-\x7F]', regex=True, na=False)

        col_mask = hex_mask | backslash_mask | non_ascii_mask
        bad_mask = bad_mask | col_mask

        print(f"  Column '{col}': {col_mask.sum():,} suspicious values")

    bad_rows = df[bad_mask]
    print(f"\n  Total rows with suspicious names: {len(bad_rows):,}")

    if len(bad_rows) == 0:
        print("  No garbled names detected.")
        return

    print(f"\n  Sample suspicious rows (up to {n_samples}):")
    for _, row in bad_rows.head(n_samples).iterrows():
        # Prefer normalized names for display, fall back to raw names
        left = str(row.get('full_name_norm_L', row.get('full_name_L', '')))
        right = str(row.get('full_name_norm_R', row.get('full_name_R', '')))
        print(f"    L: '{left}'  |  R: '{right}'")

inspect_garbled_names(cand_lr_nonid, "cand_lr_nonid")


=== cand_lr_nonid: garbled name check ===
  Column 'full_name_norm_L': 0 suspicious values
  Column 'full_name_norm_R': 0 suspicious values

  Total rows with suspicious names: 0
  No garbled names detected.


In [69]:
# Quick counts for NON-ID candidates
for name, df in [('LR_nonid', cand_lr_nonid), ('LS_nonid', cand_ls_nonid)]:
    n = len(df)
    nonid = (df['block_type'] == 'non_id_eq').sum() if 'block_type' in df.columns else 0
    print(f'{name} candidates: total={n}  non_id_eq={nonid}')

LR_nonid candidates: total=15879  non_id_eq=15879
LS_nonid candidates: total=6965  non_id_eq=6965


## 5. Compute similarity scores


  ### 5.1 Define comparators

In [71]:
# Compute similarity with PyDI RuleBasedMatcher (name comparators)
# Note: All imports are in Section 0.4

# Define comparators (name-only to avoid custom functions)
name_comparators = [
    StringComparator(
        column="full_name_normalized" if "full_name_normalized" in L.columns else "full_name",
        similarity_function="levenshtein",
        preprocess=str.lower,
    ),
    StringComparator(
        column="full_name_normalized" if "full_name_normalized" in L.columns else "full_name",
        similarity_function="jaccard",
        tokenization="word",
        preprocess=str.lower,
    ),
]
name_weights = [0.7, 0.3]  # emphasize Levenshtein


### 5.2 Score candidates with PyDI RuleBasedMatcher

In [72]:
# Compute similarity with PyDI (NON-ID channel, use pre-generated candidates; ignore team)
# Note: Uses shared utility function merge_candidate_columns from Section 0.3

# Reuse name_comparators and name_weights defined above

def score_edge_with_pydi_nonid(candidates_df: pd.DataFrame, df_left: pd.DataFrame, df_right: pd.DataFrame, left_tag: str, right_tag: str) -> pd.DataFrame:
    """Score NON-ID candidates using RuleBasedMatcher on name-only comparators.
    - Uses pre-generated candidate pairs (from build_nonid_candidates) to avoid duplicate blocking
    - Candidates should have 'id1' and 'id2' columns matching _rid values
    - Ignore team; no same_team column
    
    Uses shared utility function: merge_candidate_columns
    """
    # Use pre-generated candidates instead of recreating blocker
    # Ensure candidates_df has required columns
    if 'id1' not in candidates_df.columns or 'id2' not in candidates_df.columns:
        raise ValueError("candidates_df must have 'id1' and 'id2' columns")
    
    matcher = RuleBasedMatcher()
    correspondences = matcher.match(
        df_left=df_left, df_right=df_right,
        candidates=candidates_df[['id1', 'id2']],  # Use pre-generated candidate pairs
        comparators=name_comparators, weights=name_weights,
        threshold=0.0, id_column='_rid'
    )

    score_alias = next((col for col in ['sim','score','similarity'] if col in correspondences.columns), None)
    if score_alias and score_alias != 'sim':
        correspondences = correspondences.rename(columns={score_alias:'sim'})

    # Use shared merge function
    c = merge_candidate_columns(correspondences, df_left, df_right, left_tag, right_tag)

    c = c.rename(columns={'sim':'similarity_score'})
    if 'season_year' in c.columns:
        c[f'season_year_{left_tag}'] = c['season_year']
        c[f'season_year_{right_tag}'] = c['season_year']
    c['block_type'] = 'non_id_eq'

    keep = [
        'id1' if 'id1' in c.columns else None,
        'id2' if 'id2' in c.columns else None,
        'similarity_score',
        f'player_id_{left_tag}' if f'player_id_{left_tag}' in c.columns else None,
        f'player_id_{right_tag}' if f'player_id_{right_tag}' in c.columns else None,
        f'season_year_{left_tag}', f'season_year_{right_tag}',
        f'full_name_{left_tag}' if f'full_name_{left_tag}' in c.columns else None,
        f'full_name_{right_tag}' if f'full_name_{right_tag}' in c.columns else None,
        f'birth_year_{left_tag}' if f'birth_year_{left_tag}' in c.columns else None,
        f'birth_year_{right_tag}' if f'birth_year_{right_tag}' in c.columns else None,
        'block_type'
    ]
    keep = [k for k in keep if k and k in c.columns]
    return c[keep]



In [73]:
# Prepare NON-ID blocking keys on L/R/S for scoring (if missing)
# Note: Uses shared utility function add_name_prefix from Section 0.3

def ensure_nonid_keys(df: pd.DataFrame, tag: str) -> None:
    """Ensure NON-ID blocking keys exist on dataframe.
    
    Uses shared utility function: create_nonid_keys
    If both keys already exist, this function does nothing (idempotent).
    If either key is missing, creates both keys to ensure consistency.
    """
    # Only create keys if either is missing (create both to ensure consistency)
    if '_key_nonid' not in df.columns or '_rid' not in df.columns:
        create_nonid_keys(df, tag, inplace=True)

# Apply to three dataframes
ensure_nonid_keys(L, 'L')
ensure_nonid_keys(R, 'R')
ensure_nonid_keys(S, 'S')

print('NON-ID keys prepared on L/R/S:', '_key_nonid' in L.columns, '_key_nonid' in R.columns, '_key_nonid' in S.columns)

NON-ID keys prepared on L/R/S: True True True


### 6.2 Cluster Consistency Analysis

Analyze the cluster structure to identify any inconsistencies that our evaluation set may miss. The `EntityMatchingEvaluator` offers the `create_cluster_size_distribution` method for this purpose.


In [74]:
# Score three edges with PyDI matcher (name comparators)
# Use pre-generated candidate pairs to avoid duplicate blocking
cand_lr = score_edge_with_pydi_nonid(cand_lr_nonid, L, R, 'L', 'R').sort_values('similarity_score', ascending=False).reset_index(drop=True)
# cand_rs = score_edge_with_pydi_nonid(cand_rs_nonid, R, S, 'R', 'S').sort_values('similarity_score', ascending=False).reset_index(drop=True)
cand_ls = score_edge_with_pydi_nonid(cand_ls_nonid, L, S, 'L', 'S').sort_values('similarity_score', ascending=False).reset_index(drop=True)

print('Similarity scores computed (PyDI) and candidates sorted')
print('LR: similarity_score range =', cand_lr['similarity_score'].min(), 'to', cand_lr['similarity_score'].max())
# print('RS: similarity_score range =', cand_rs['similarity_score'].min(), 'to', cand_rs['similarity_score'].max())
print('LS: similarity_score range =', cand_ls['similarity_score'].min(), 'to', cand_ls['similarity_score'].max())


[INFO ] PyDI.entitymatching.rule_based.RuleBasedMatcher - Starting Entity Matching
[INFO ] PyDI.entitymatching.rule_based.RuleBasedMatcher - Blocking 106553 x 15215 elements
[INFO ] PyDI.entitymatching.rule_based.RuleBasedMatcher - Matching 106553 x 15215 elements after 0:00:0.126; 15879 blocked pairs (reduction ratio: 0.9999902054269367)
[INFO ] PyDI.entitymatching.rule_based.RuleBasedMatcher - Entity Matching finished after 0:00:1.593; found 15879 correspondences.
[INFO ] PyDI.entitymatching.rule_based.RuleBasedMatcher - Starting Entity Matching
[INFO ] PyDI.entitymatching.rule_based.RuleBasedMatcher - Blocking 106553 x 6743 elements
[INFO ] PyDI.entitymatching.rule_based.RuleBasedMatcher - Matching 106553 x 6743 elements after 0:00:0.091; 6965 blocked pairs (reduction ratio: 0.9999903060164304)
[INFO ] PyDI.entitymatching.rule_based.RuleBasedMatcher - Entity Matching finished after 0:00:0.518; found 6965 correspondences.


Similarity scores computed (PyDI) and candidates sorted
LR: similarity_score range = 0.19999999999999998 to 1.0
LS: similarity_score range = 0.19999999999999998 to 1.0


  ### 5.3 Inspect similarity distribution


In [75]:
# Print similarity distribution statistics
for name, df in [('LR', cand_lr), ('LS', cand_ls)]:
    print(f'\n{name} similarity distribution:')
    print(f'  mean: {df["similarity_score"].mean():.3f}')
    print(f'  median: {df["similarity_score"].median():.3f}')
    print(f'  min: {df["similarity_score"].min():.3f}')
    print(f'  max: {df["similarity_score"].max():.3f}')
    print(f'  std: {df["similarity_score"].std():.3f}')
    
    # Percentiles for stratified sampling
    print('  percentiles:')
    for p in [10, 25, 50, 75, 90]:
        val = df['similarity_score'].quantile(p/100)
        print(f'    {p}th: {val:.3f}')



LR similarity distribution:
  mean: 0.979
  median: 1.000
  min: 0.200
  max: 1.000
  std: 0.101
  percentiles:
    10th: 1.000
    25th: 1.000
    50th: 1.000
    75th: 1.000
    90th: 1.000

LS similarity distribution:
  mean: 0.981
  median: 1.000
  min: 0.200
  max: 1.000
  std: 0.097
  percentiles:
    10th: 1.000
    25th: 1.000
    50th: 1.000
    75th: 1.000
    90th: 1.000


## 6. Stratified sampling for annotation


In [76]:
MANUAL_CASES_DIR = OUTPUT_DIR / 'gt' / 'manual_cases'
MANUAL_CASES_DIR.mkdir(parents=True, exist_ok=True)

MANUAL_ERROR_CASES = defaultdict(set)
for edge in ['LR', 'LS']:
    manual_path = MANUAL_CASES_DIR / f'manual_cases_{edge}.csv'
    if manual_path.exists():
        manual_df = pd.read_csv(manual_path)
        if {'id1', 'id2'}.issubset(manual_df.columns):
            MANUAL_ERROR_CASES[edge] = set(zip(manual_df['id1'], manual_df['id2']))


def _strip_accents(text: str) -> str:
    if not isinstance(text, str):
        return ''
    normalized = unicodedata.normalize('NFD', text)
    return ''.join(ch for ch in normalized if unicodedata.category(ch) != 'Mn')


def _has_accent_or_suffix_variant(row: pd.Series, left_tag: str, right_tag: str) -> bool:
    left = str(row.get(f'full_name_{left_tag}', '')).lower().strip()
    right = str(row.get(f'full_name_{right_tag}', '')).lower().strip()
    if not left or not right:
        return False
    accent_left = _strip_accents(left)
    accent_right = _strip_accents(right)
    if accent_left and accent_left == accent_right and left != right:
        return True
    suffix_tokens = {'jr', 'sr', 'ii', 'iii', 'iv', 'v'}
    left_suffix = any(left.endswith(f' {s}') for s in suffix_tokens)
    right_suffix = any(right.endswith(f' {s}') for s in suffix_tokens)
    if left_suffix != right_suffix:
        def _strip_suffix(text: str) -> str:
            parts = [tok for tok in text.split() if tok not in suffix_tokens]
            return ' '.join(parts)
        base_left = _strip_accents(_strip_suffix(left))
        base_right = _strip_accents(_strip_suffix(right))
        return base_left == base_right
    return False


def _has_birth_conflict(row: pd.Series, left_tag: str, right_tag: str, threshold: int = 2) -> bool:
    by_l = pd.to_numeric(row.get(f'birth_year_{left_tag}'), errors='coerce')
    by_r = pd.to_numeric(row.get(f'birth_year_{right_tag}'), errors='coerce')
    if pd.isna(by_l) or pd.isna(by_r):
        return False
    return abs(by_l - by_r) >= threshold



### 6.1 Build stratified samples

In [77]:
# Build stratified samples for annotation (score-threshold bucketing)

SAMPLE_OUT_DIR = OUTPUT_DIR / 'gt' / 'samples'
SAMPLE_OUT_DIR.mkdir(parents=True, exist_ok=True)


def ensure_ids_for_annotation(df: pd.DataFrame, edge: str) -> pd.DataFrame:
    left_tag, right_tag = edge[0], edge[1]
    c = df.copy()
    # If id1/id2 already exist, keep them
    if 'id1' in c.columns and 'id2' in c.columns:
        return c
    # Build synthetic IDs from available columns (full_name + season_year), fallback to index
    l_name = f'full_name_{left_tag}' if f'full_name_{left_tag}' in c.columns else None
    r_name = f'full_name_{right_tag}' if f'full_name_{right_tag}' in c.columns else None
    l_year_col = f'season_year_{left_tag}'
    r_year_col = f'season_year_{right_tag}'
    l_part = (c[l_name].astype(str) if l_name else c.index.astype(str))
    r_part = (c[r_name].astype(str) if r_name else c.index.astype(str))
    c['id1'] = l_part.str.strip().str.lower() + '|' + pd.to_numeric(c[l_year_col], errors='coerce').astype('Int64').astype(str) + f'|{left_tag}'
    c['id2'] = r_part.str.strip().str.lower() + '|' + pd.to_numeric(c[r_year_col], errors='coerce').astype('Int64').astype(str) + f'|{right_tag}'
    return c


def stratified_sample_qcut(cand: pd.DataFrame, edge: str, n_total: int = 500,
                           high_ratio: float = 0.20, corner_ratio: float = 0.30,
                           seed: int = 42) -> pd.DataFrame:
    df = ensure_ids_for_annotation(cand, edge)
    # Score-threshold bucketing: low < 0.60, mid in [0.60, 0.99), high >= 0.99
    scores = pd.to_numeric(df['similarity_score'], errors='coerce')
    df['bucket'] = pd.cut(scores, bins=[-1, 0.60, 0.99, 1.01], labels=['low','mid','high'], include_lowest=True)
    high = df[df['bucket']=='high']
    mid  = df[df['bucket']=='mid']
    low  = df[df['bucket']=='low']

    # corner: boundary + hard positives + hard negatives (prefer sampling from mid bucket; backfill from all if insufficient)
    l_tag, r_tag = edge[0], edge[1]
    by_l, by_r = f'birth_year_{l_tag}', f'birth_year_{r_tag}'
    sy_l, sy_r = f'season_year_{l_tag}', f'season_year_{r_tag}'
    pid_l, pid_r = f'player_id_{l_tag}', f'player_id_{r_tag}'

    # 1) Boundary samples: near-threshold ambiguous region (adjust the windows if needed)
    boundary = df[
        scores.between(0.58, 0.62, inclusive='both') |
        scores.between(0.98, 0.99, inclusive='left')
    ]

    # 2) Hard positives: same player_id but conflicting attributes and/or low similarity
    same_pid = False
    if (pid_l in df.columns) and (pid_r in df.columns):
        same_pid = (df[pid_l].astype(str).str.lower() == df[pid_r].astype(str).str.lower())
    birth_diff = None
    if (by_l in df.columns) and (by_r in df.columns):
        birth_diff = (pd.to_numeric(df[by_l], errors='coerce') - pd.to_numeric(df[by_r], errors='coerce')).abs().gt(0)
    hard_pos = df[same_pid] if isinstance(same_pid, pd.Series) else df.head(0)
    if len(hard_pos) > 0:
        idx = hard_pos.index
        cond_low = scores.loc[idx] < 0.80
        cond_birth = birth_diff.loc[idx] if isinstance(birth_diff, pd.Series) else pd.Series(False, index=idx)
        hard_pos = hard_pos[cond_low | cond_birth.fillna(False)]

    # 3) Hard negatives: different player_id but highly similar attributes (same season and birth year) with high similarity
    diff_pid = False
    if (pid_l in df.columns) and (pid_r in df.columns):
        diff_pid = (df[pid_l].astype(str).str.lower() != df[pid_r].astype(str).str.lower())
    same_season = False
    if (sy_l in df.columns) and (sy_r in df.columns):
        same_season = (pd.to_numeric(df[sy_l], errors='coerce') == pd.to_numeric(df[sy_r], errors='coerce'))
    birth_eq = False
    if (by_l in df.columns) and (by_r in df.columns):
        birth_eq = (pd.to_numeric(df[by_l], errors='coerce') == pd.to_numeric(df[by_r], errors='coerce'))
    mask_hn = (diff_pid if isinstance(diff_pid, pd.Series) else False)
    if isinstance(mask_hn, pd.Series):
        if isinstance(same_season, pd.Series):
            mask_hn = mask_hn & same_season
        if isinstance(birth_eq, pd.Series):
            mask_hn = mask_hn & birth_eq
        mask_hn = mask_hn & scores.ge(0.95)
        hard_neg = df[mask_hn]
    else:
        hard_neg = df.head(0)

    # Merge corner candidates; prefer mid bucket; backfill from all candidates if needed
    corner_all = pd.concat([boundary, hard_pos, hard_neg], axis=0).drop_duplicates()

    # Pattern boosters: manual error pairs, accent/suffix variants, and large birth-year conflicts
    pattern_mask = None
    manual_pairs = MANUAL_ERROR_CASES.get(edge, set())
    if manual_pairs:
        manual_mask = df.apply(lambda r: (r['id1'], r['id2']) in manual_pairs, axis=1)
        pattern_mask = manual_mask if pattern_mask is None else (pattern_mask | manual_mask)

    accent_mask = df.apply(lambda r: _has_accent_or_suffix_variant(r, l_tag, r_tag), axis=1)
    birth_conflict_mask = df.apply(lambda r: _has_birth_conflict(r, l_tag, r_tag), axis=1)
    for mask in [accent_mask, birth_conflict_mask]:
        if isinstance(mask, pd.Series):
            pattern_mask = mask if pattern_mask is None else (pattern_mask | mask)

    if pattern_mask is not None:
        pattern_cases = df[pattern_mask]
        if len(pattern_cases) > 0:
            corner_all = pd.concat([corner_all, pattern_cases], axis=0).drop_duplicates()

    corner_mid = corner_all[corner_all.index.isin(mid.index)]
    corner = corner_mid
    need = int(n_total*corner_ratio) - len(corner)
    if need > 0:
        extra = corner_all[~corner_all.index.isin(corner.index)]
        corner = pd.concat([corner, extra.head(max(0, need))])

    n_high = min(int(n_total*high_ratio), len(high))
    n_corner = min(int(n_total*corner_ratio), len(corner))
    n_low = min(n_total - n_high - n_corner, len(low))

    sampled = pd.concat([
        high.sample(n=n_high, random_state=seed) if n_high>0 else high.head(0),
        corner.sample(n=n_corner, random_state=seed) if n_corner>0 else corner.head(0),
        low.sample(n=n_low, random_state=seed) if n_low>0 else low.head(0),
    ], ignore_index=True)

       # Backfill if total < n_total: prefer mid -> high -> low from remaining candidates
    need = n_total - len(sampled)
    if need > 0:
        # Build a key to avoid duplicates (id1,id2)
        chosen_keys = set(zip(sampled['id1'], sampled['id2']))
        remaining = df[~list(zip(df['id1'], df['id2'])) \
                          .__iter__().__class__(zip(df['id1'], df['id2'])) \
                      ] if False else df  # placeholder; see below

        # Robust way to filter remaining by keys without heavy memory:
        rem_mask = ~df.apply(lambda r: (r['id1'], r['id2']) in chosen_keys, axis=1)
        remaining = df[rem_mask]

        # Priority pools
        pools = [
            remaining[remaining['bucket'] == 'mid'],
            remaining[remaining['bucket'] == 'high'],
            remaining[remaining['bucket'] == 'low'],
        ]
        for pool in pools:
            if need <= 0 or len(pool) == 0:
                continue
            take = min(need, len(pool))
            add = pool.sample(n=take, random_state=seed)
            sampled = pd.concat([sampled, add], ignore_index=True)
            # update chosen set to keep uniqueness in later loops
            for t in zip(add['id1'], add['id2']):
                chosen_keys.add(t)
            need -= take

    cols = [
        'id1','id2','similarity_score',
        f'full_name_{edge[0]}' if f'full_name_{edge[0]}' in sampled.columns else None,
        f'full_name_{edge[1]}' if f'full_name_{edge[1]}' in sampled.columns else None,
        f'season_year_{edge[0]}', f'season_year_{edge[1]}',
        f'birth_year_{edge[0]}' if f'birth_year_{edge[0]}' in sampled.columns else None,
        f'birth_year_{edge[1]}' if f'birth_year_{edge[1]}' in sampled.columns else None,
    ]
    cols = [c for c in cols if c and c in sampled.columns]
    df_out = sampled[cols].copy()
    # Prepare empty label column for manual annotation
    df_out['label'] = ''
    # Mark candidate source channel for evaluation stratification
    df_out['source_channel'] = 'non_id'
    return df_out


# Apply to three edges and save (NON-ID channel, use scored candidates with similarity_score)
edges = {
    'LR': ('cand_lr', 'L', 'R'),
    #'RS': ('cand_rs', 'R', 'S'),
    'LS': ('cand_ls', 'L', 'S'),
}

results = {}
for edge_name, (var_name, lt, rt) in edges.items():
    cand_df = globals()[var_name]
    sampled = stratified_sample_qcut(cand_df, edge=f'{lt}{rt}', n_total=500)
    results[edge_name] = sampled
    out_path = SAMPLE_OUT_DIR / f'samples_{edge_name}_v1.csv'
    sampled.to_csv(out_path, index=False)
    print(f'{edge_name}: saved {len(sampled)} rows to', out_path)

for edge_name, dfp in results.items():
    print(f"\n{edge_name} sample preview:")
    display(dfp.head(5))


LR: saved 500 rows to /Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/gt/samples/samples_LR_v1.csv
LS: saved 500 rows to /Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/gt/samples/samples_LS_v1.csv

LR sample preview:


,id1,id2,similarity_score,full_name_L,full_name_R,season_year_L,season_year_R,birth_year_L,birth_year_R,label,source_channel
0,4613142010|2010|L,4613142010|2010|R,1.0,matt kemp,matt kemp,2010,2010,1984,1985,,non_id
1,3467982015|2015|L,3467982015|2015|R,1.0,kyle lohse,kyle lohse,2015,2015,1978,1979,,non_id
2,6135642021|2021|L,6135642021|2021|R,1.0,jason vosler,jason vosler,2021,2021,1993,1994,,non_id
3,5379532012|2012|L,5379532012|2012|R,1.0,daniel nava,daniel nava,2012,2012,1983,1983,,non_id
4,6053232015|2015|L,6053232015|2015|R,1.0,kyle kubitza,kyle kubitza,2015,2015,1990,1991,,non_id



LS sample preview:


,id1,id2,similarity_score,full_name_L,full_name_S,season_year_L,season_year_S,birth_year_L,birth_year_S,label,source_channel
0,4346582019|2019|L,4346582019|2019|S,1.0,rajai davis,rajai davis,2019,2019,1980,1981,,non_id
1,5423642022|2022|L,5423642022|2022|S,1.0,rafael ortega,rafael ortega,2022,2022,1991,1991,,non_id
2,6083282021|2021|L,6083282021|2021|S,1.0,chase de jong,chase de jong,2021,2021,1993,1994,,non_id
3,5785702023|2023|L,5785702023|2023|S,1.0,juniel querecuto,juniel querecuto,2023,2023,1992,1993,,non_id
4,5431482017|2017|L,5431482017|2017|S,1.0,tim federowicz,tim federowicz,2017,2017,1987,1988,,non_id


In [78]:

def simple_cases(df: pd.DataFrame, edge: str, k: int = 10):
    lt, rt = edge[0], edge[1]
    cols = [
        'id1','id2','similarity_score',
        f'full_name_{lt}' if f'full_name_{lt}' in df.columns else None,
        f'full_name_{rt}' if f'full_name_{rt}' in df.columns else None,
        f'season_year_{lt}', f'season_year_{rt}',
        f'birth_year_{lt}' if f'birth_year_{lt}' in df.columns else None,
        f'birth_year_{rt}' if f'birth_year_{rt}' in df.columns else None,
    ]
    cols = [c for c in cols if c and c in df.columns]

    d = df.copy()
    scores = pd.to_numeric(d['similarity_score'], errors='coerce')
    d['bucket'] = pd.cut(scores, bins=[-1, 0.60, 0.99, 1.01], labels=['low','mid','high'], include_lowest=True)

    print(f'\n=== {edge} HIGH (top 10) ===')
    hi = d[d['bucket']=='high'].sort_values('similarity_score', ascending=False).head(k)
    display(hi[cols])

    print(f'\n=== {edge} MID (10 random) ===')
    mid = d[d['bucket']=='mid']
    mid = (mid.sample(n=min(k, len(mid)), random_state=42) if len(mid) > k else mid.head(k))
    display(mid.sort_values('similarity_score', ascending=False)[cols])

    print(f'\n=== {edge} LOW (bottom 10) ===')
    lo = d[d['bucket']=='low'].sort_values('similarity_score', ascending=True).head(k)
    display(lo[cols])


# Run for three edges using NON-ID scored candidates (cand_lr/cand_rs/cand_ls)
simple_cases(cand_lr, 'LR', k=5)
# simple_cases(cand_rs, 'RS', k=5)
simple_cases(cand_ls, 'LS', k=5)


=== LR HIGH (top 10) ===


,id1,id2,similarity_score,full_name_L,full_name_R,season_year_L,season_year_R,birth_year_L,birth_year_R
0,1100292008|2008|L,1100292008|2008|R,1.0,bobby abreu,bobby abreu,2008,2008,1974,1974
10129,1504392008|2008|L,1504392008|2008|R,1.0,gary matthews,gary matthews,2008,2008,1974,1975
10117,1504152008|2008|L,1504152008|2008|R,1.0,brad wilkerson,brad wilkerson,2008,2008,1977,1977
10118,1504162008|2008|L,1504162008|2008|R,1.0,danny ardoin,danny ardoin,2008,2008,1974,1975
10119,1504212008|2008|L,1504212008|2008|R,1.0,ramon hernandez,ram\xc3\xb3n hern\xc3\xa1ndez,2008,2008,1976,1976



=== LR MID (10 random) ===


,id1,id2,similarity_score,full_name_L,full_name_R,season_year_L,season_year_R,birth_year_L,birth_year_R
15241,5759292021|2021|L,6613882021|2021|R,0.676471,willson contreras,william contreras,2021,2021,1992,1998
15291,1120202010|2010|L,4520802010|2010|R,0.660000,chris carpenter,chris carter,2010,2010,1975,1983
15294,1120202010|2010|L,4748922010|2010|R,0.660000,chris carpenter,chris carter,2010,2010,1975,1987
15280,2361552011|2011|L,4296642011|2011|R,0.660000,robinson cancel,robinson can\xc3\xb3,2011,2011,1976,1983
15328,5925272016|2016|L,4928412016|2016|R,0.625000,michael mariot,michael mart\xc3\xadnez,2016,2016,1988,1983



=== LR LOW (bottom 10) ===


...

,id1,id2,similarity_score,full_name_L,full_name_S,season_year_L,season_year_S,birth_year_L,birth_year_S
0,1125262015|2015|L,1125262015|2015|S,1.0,bartolo colon,bartolo colon,2015,2015,1973,1973
4385,4448762017|2017|L,4448762017|2017|S,1.0,alcides escobar,alcides escobar,2017,2017,1986,1987
4461,4444822015|2015|L,4444822015|2015|S,1.0,david peralta,david peralta,2015,2015,1987,1988
4460,4444822016|2016|L,4444822016|2016|S,1.0,david peralta,david peralta,2016,2016,1987,1988
4459,4444822017|2017|L,4444822017|2017|S,1.0,david peralta,david peralta,2017,2017,1987,1988



=== LS MID (10 random) ===


,id1,id2,similarity_score,full_name_L,full_name_S,season_year_L,season_year_S,birth_year_L,birth_year_S
6681,5471702019|2019|L,5471702019|2019|S,0.753333,nick delmonico,nicky delmonico,2019,2019,1992,1993
6693,6689422022|2022|L,6703512022|2022|S,0.730000,josh rojas,jose rojas,2022,2022,1994,1993
6703,5948382019|2019|L,5948382019|2019|S,0.706667,philip gosselin,phil gosselin,2019,2019,1988,1989
6756,6647472022|2022|L,6495572022|2022|S,0.625000,alexis diaz,aledmys diaz,2022,2022,1996,1991
6751,6072572017|2017|L,6072572017|2017|S,0.625000,rafael lopez,raffy lopez,2017,2017,1987,1988



=== LS LOW (bottom 10) ===


,id1,id2,similarity_score,full_name_L,full_name_S,season_year_L,season_year_S,birth_year_L,birth_year_S
6964,6723862022|2022|L,6661352022|2022|S,0.2,alejandro kirk,alex kirilloff,2022,2022,1998,1998
6957,6661352023|2023|L,6723862023|2023|S,0.2,alex kirilloff,alejandro kirk,2023,2023,1997,1999
6958,6723862023|2023|L,6661352023|2023|S,0.2,alejandro kirk,alex kirilloff,2023,2023,1998,1998
6959,6661352022|2022|L,6723862022|2022|S,0.2,alex kirilloff,alejandro kirk,2022,2022,1997,1999
6963,6723862024|2024|L,6661352024|2024|S,0.2,alejandro kirk,alex kirilloff,2024,2024,1998,1998


In [79]:
def summarize_patterns(sample_df, edge):
    lt, rt = edge[0], edge[1]
    manual_pairs = MANUAL_ERROR_CASES.get(edge, set())

    def _is_manual(row):
        return (row['id1'], row['id2']) in manual_pairs

    summary = {
        'manual_cases': sample_df.apply(_is_manual, axis=1).sum(),
        'accent_suffix': sample_df.apply(lambda r: _has_accent_or_suffix_variant(r, lt, rt), axis=1).sum(),
        'birth_conflict': sample_df.apply(lambda r: _has_birth_conflict(r, lt, rt), axis=1).sum(),
    }
    print(f'{edge} pattern counts:', summary)
    return summary

summarize_patterns(results['LR'], 'LR')
summarize_patterns(results['LS'], 'LS')

LR pattern counts: {'manual_cases': np.int64(5), 'accent_suffix': np.int64(1), 'birth_conflict': np.int64(292)}
LS pattern counts: {'manual_cases': np.int64(9), 'accent_suffix': np.int64(5), 'birth_conflict': np.int64(205)}


{'manual_cases': np.int64(9),
 'accent_suffix': np.int64(5),
 'birth_conflict': np.int64(205)}

### 6.2 Auto-labeling with conservative rules

In [80]:
# Conservative prelabel: only update existing 'label' column (no extra columns)

SAMPLE_OUT_DIR = OUTPUT_DIR / 'gt' / 'samples'
for edge in ['LR','LS']:
    in_path = SAMPLE_OUT_DIR / f'samples_{edge}_v1.csv'
    if not in_path.exists():
        print('MISS:', in_path)
        continue
    df = pd.read_csv(in_path)
    if 'label' not in df.columns:
        df['label'] = ''
    df['label'] = df['label'].astype('string').fillna('')

    lt, rt = edge[0], edge[1]
    sy_l, sy_r = f'season_year_{lt}', f'season_year_{rt}'
    by_l, by_r = f'birth_year_{lt}', f'birth_year_{rt}'

    score = pd.to_numeric(df['similarity_score'], errors='coerce')
    y_eq = (pd.to_numeric(df.get(sy_l), errors='coerce') == pd.to_numeric(df.get(sy_r), errors='coerce'))
    by_diff = (pd.to_numeric(df.get(by_l), errors='coerce') - pd.to_numeric(df.get(by_r), errors='coerce')).abs()

    # compare ids ignoring the trailing side tag (|L/|R/|S)
    # Extract core IDs using shared utility function
    core1, core2 = extract_core_ids(df)
    same_core = core1 == core2

    empty = (df['label'] == '')
    # TRUE: core(id1) == core(id2)
    fill_true = empty & same_core
    df.loc[fill_true, 'label'] = 'TRUE'

    # FALSE: for remaining empty labels where core(id1) != core(id2)
    fill_false = empty & (~same_core)
    df.loc[fill_false, 'label'] = 'FALSE'

    out_path = SAMPLE_OUT_DIR / f'samples_{edge}_v1_prelabel_pid.csv'
    df.to_csv(out_path, index=False)
    print(edge, 'prelabeled ->', out_path, 'TRUE=', (df['label']=='TRUE').sum(), 'FALSE=', (df['label']=='FALSE').sum(), 'REVIEW=', (df['label']=='').sum())



LR prelabeled -> /Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/gt/samples/samples_LR_v1_prelabel_pid.csv TRUE= 150 FALSE= 350 REVIEW= 0
LS prelabeled -> /Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/gt/samples/samples_LS_v1_prelabel_pid.csv TRUE= 238 FALSE= 262 REVIEW= 0


## 7. Ground-truth quality checks

In [81]:
# This cell evaluates the labeled ground-truth CSVs using simple, interpretable metrics.
# It checks: basic counts, label distribution, similarity-by-label stats, core-ID consistency,
# and potential labeling inconsistencies (e.g., high-similarity FALSE, low-similarity TRUE).

SAMPLE_OUT_DIR = OUTPUT_DIR / 'gt' / 'samples'


def _load_gt(edge: str) -> pd.DataFrame:
    path = SAMPLE_OUT_DIR / f'samples_{edge}_v1_prelabel_pid.csv'
    if not path.exists():
        print(f"MISSING: {path}")
        return pd.DataFrame()
    df = pd.read_csv(path)
    # Ensure expected columns exist
    for c in ['id1','id2','similarity_score','label']:
        if c not in df.columns:
            df[c] = pd.NA
    return df


# Note: Uses shared utility function extract_core_ids from Section 0.3

def assess_edge(df: pd.DataFrame, edge: str) -> dict:
    """Assess ground truth quality and return anomaly cases for detailed inspection.
    
    Returns:
        dict with keys:
            - 'high_similarity_false': DataFrame of FALSE cases with similarity >= 0.99
            - 'low_similarity_true': DataFrame of TRUE cases with similarity < 0.60
    """
    if df.empty:
        return {'high_similarity_false': pd.DataFrame(), 'low_similarity_true': pd.DataFrame()}
    
    print("\n" + "="*60)
    print(f"Ground Truth Assessment - {edge}")
    print("="*60)

    # Basic totals (normalize labels first)
    n = len(df)
    lab = (df.get('label')
           .astype('string')
           .fillna('')
           .str.strip()
           .str.upper())
    n_true = int((lab == 'TRUE').sum())
    n_false = int((lab == 'FALSE').sum())
    n_empty = int((~lab.isin(['TRUE','FALSE'])).sum())
    print(f"Total: {n}")
    print(f"TRUE: {n_true} ({(n_true/max(n,1))*100:.1f}%)  FALSE: {n_false} ({(n_false/max(n,1))*100:.1f}%)  EMPTY: {n_empty}")

    # Similarity distributions by label
    df['similarity_score'] = pd.to_numeric(df['similarity_score'], errors='coerce')
    for lab_name in ['TRUE','FALSE']:
        sub = df.loc[lab == lab_name, 'similarity_score'].dropna()
        if len(sub) == 0:
            continue
        print(f"\nSimilarity statistics for {lab_name}:")
        print(f"  mean={sub.mean():.3f}  median={sub.median():.3f}  min={sub.min():.3f}  max={sub.max():.3f}  std={sub.std():.3f}")

    # Core-ID consistency (using shared utility function)
    core1, core2 = extract_core_ids(df)
    same_core = core1 == core2
    n_true_same = int(((lab == 'TRUE') & same_core).sum())
    n_true_diff = int(((lab == 'TRUE') & ~same_core).sum())
    n_false_same = int(((lab == 'FALSE') & same_core).sum())
    n_false_diff = int(((lab == 'FALSE') & ~same_core).sum())
    print("\nCore-ID consistency:")
    print(f"  TRUE with same core-id: {n_true_same}")
    print(f"  TRUE with diff core-id: {n_true_diff}")
    print(f"  FALSE with same core-id: {n_false_same}")
    print(f"  FALSE with diff core-id: {n_false_diff}")

    # Potential inconsistencies to review (calculate and store DataFrames)
    high_similarity_false = df[(lab == 'FALSE') & (df['similarity_score'] >= 0.99)].copy()
    low_similarity_true = df[(lab == 'TRUE') & (df['similarity_score'] < 0.60)].copy()
    
    hi_false = len(high_similarity_false)
    lo_true = len(low_similarity_true)
    print("\nPotential issues (for manual review):")
    print(f"  High-similarity FALSE (score >= 0.99): {hi_false}")
    print(f"  Low-similarity TRUE (score < 0.60): {lo_true}")

    # Hard negatives (diff core-id, high similarity)
    hard_neg = int(((lab == 'FALSE') & (df['similarity_score'] >= 0.95)).sum())
    print(f"  Hard negatives (diff core-id, score >= 0.95): {hard_neg}")

    # Hard positives (same core-id, but lower similarity)
    hard_pos = int(((lab == 'TRUE') & (df['similarity_score'] < 0.80)).sum())
    print(f"  Hard positives (same core-id, score < 0.80): {hard_pos}")

    # Duplicate checks on pair keys
    pair_dups = int(df.duplicated(subset=['id1','id2'], keep=False).sum())
    print("\nDuplicate checks:")
    print(f"  Rows in duplicate (id1,id2) groups: {pair_dups}")
    
    # Return anomaly cases for detailed inspection
    return {
        'high_similarity_false': high_similarity_false,
        'low_similarity_true': low_similarity_true
    }


# Run assessment for LR and LS and store anomaly cases
lr = _load_gt('LR')
ls = _load_gt('LS')
lr_anomalies = assess_edge(lr, 'LR')
ls_anomalies = assess_edge(ls, 'LS')
print("\nDone.")




Ground Truth Assessment - LR
Total: 500
TRUE: 150 (30.0%)  FALSE: 350 (70.0%)  EMPTY: 0

Similarity statistics for TRUE:
  mean=0.889  median=1.000  min=0.550  max=1.000  std=0.159

Similarity statistics for FALSE:
  mean=0.530  median=0.538  min=0.200  max=1.000  std=0.112

Core-ID consistency:
  TRUE with same core-id: 150
  TRUE with diff core-id: 0
  FALSE with same core-id: 0
  FALSE with diff core-id: 350

Potential issues (for manual review):
  High-similarity FALSE (score >= 0.99): 1
  Low-similarity TRUE (score < 0.60): 5
  Hard negatives (diff core-id, score >= 0.95): 1
  Hard positives (same core-id, score < 0.80): 51

Duplicate checks:
  Rows in duplicate (id1,id2) groups: 18

Ground Truth Assessment - LS
Total: 500
TRUE: 238 (47.6%)  FALSE: 262 (52.4%)  EMPTY: 0

Similarity statistics for TRUE:
  mean=0.941  median=1.000  min=0.550  max=1.000  std=0.127

Similarity statistics for FALSE:
  mean=0.510  median=0.513  min=0.200  max=1.000  std=0.122

Core-ID consistency:
  TR

In [82]:
# 7. Detailed inspection of potential issues
# Check LS high-similarity FALSE and LR low-similarity TRUE cases
# Use anomaly cases from assess_edge (Cell 57) - ensure Cell 57 has been executed
# If anomalies are not available, fall back to loading and filtering
if 'ls_anomalies' not in globals() or 'lr_anomalies' not in globals():
    print("Warning: Anomaly cases not found. Please run Cell 57 (assess_edge) first.")
    print("Falling back to loading and filtering...")
    SAMPLE_OUT_DIR = OUTPUT_DIR / 'gt' / 'samples'
    df_lr = _load_gt('LR')
    df_ls = _load_gt('LS')
    for df in [df_lr, df_ls]:
        df['label_norm'] = df['label'].astype('string').fillna('').str.strip().str.upper()
        df['similarity_score'] = pd.to_numeric(df['similarity_score'], errors='coerce')
    ls_high_false = df_ls[(df_ls['label_norm'] == 'FALSE') & (df_ls['similarity_score'] >= 0.99)]
    lr_low_true = df_lr[(df_lr['label_norm'] == 'TRUE') & (df_lr['similarity_score'] < 0.60)]
else:
    # Use pre-calculated anomaly cases from assess_edge
    ls_high_false = ls_anomalies['high_similarity_false'].copy()
    lr_low_true = lr_anomalies['low_similarity_true'].copy()
    # Normalize labels for consistency
    ls_high_false['label_norm'] = ls_high_false['label'].astype('string').fillna('').str.strip().str.upper()
    lr_low_true['label_norm'] = lr_low_true['label'].astype('string').fillna('').str.strip().str.upper()

print("="*80)
print("1. LS: High-similarity FALSE cases (similarity >= 0.99)")
print("="*80)
print(f"Found {len(ls_high_false)} cases\n")

if len(ls_high_false) > 0:
    # Display all columns for inspection
    cols_to_show = ['id1', 'id2', 'similarity_score', 'full_name_L', 'full_name_S', 
                    'season_year_L', 'season_year_S', 'birth_year_L', 'birth_year_S', 'label']
    display_cols = [c for c in cols_to_show if c in ls_high_false.columns]
    
    print("Full details:")
    display(ls_high_false[display_cols])
    
    # Extract core IDs for comparison (using shared utility function)
    core1, core2 = extract_core_ids(ls_high_false)
    print("\nCore ID comparison:")
    for idx, row in ls_high_false.iterrows():
        print(f"  Row {idx}: core1='{core1.loc[idx]}' vs core2='{core2.loc[idx]}' (match: {core1.loc[idx] == core2.loc[idx]})")
        print(f"    Name L: '{row.get('full_name_L', 'N/A')}'")
        print(f"    Name S: '{row.get('full_name_S', 'N/A')}'")
        print(f"    Season: {row.get('season_year_L', 'N/A')} vs {row.get('season_year_S', 'N/A')}")
        print(f"    Birth: {row.get('birth_year_L', 'N/A')} vs {row.get('birth_year_S', 'N/A')}")
        print()

print("\n" + "="*80)
print("2. LR: Low-similarity TRUE cases (similarity < 0.60)")
print("="*80)
print(f"Found {len(lr_low_true)} cases\n")

if len(lr_low_true) > 0:
    # Show summary statistics
    print("Summary statistics:")
    print(f"  Similarity range: [{lr_low_true['similarity_score'].min():.3f}, {lr_low_true['similarity_score'].max():.3f}]")
    print(f"  Mean similarity: {lr_low_true['similarity_score'].mean():.3f}")
    print(f"  Median similarity: {lr_low_true['similarity_score'].median():.3f}")
    
    # Check birth year differences
    by_diff = (pd.to_numeric(lr_low_true.get('birth_year_L'), errors='coerce') - 
               pd.to_numeric(lr_low_true.get('birth_year_R'), errors='coerce')).abs()
    print(f"\n  Birth year differences:")
    print(f"    Same year: {(by_diff == 0).sum()} cases")
    print(f"    Diff by 1: {(by_diff == 1).sum()} cases")
    print(f"    Diff by 2+: {(by_diff > 1).sum()} cases")
    print(f"    Missing: {by_diff.isna().sum()} cases")
    
    # Check season year matches
    sy_match = (pd.to_numeric(lr_low_true.get('season_year_L'), errors='coerce') == 
                pd.to_numeric(lr_low_true.get('season_year_R'), errors='coerce'))
    print(f"\n  Season year matches: {sy_match.sum()} / {len(lr_low_true)}")
    
    # Show first 20 cases for detailed inspection
    print("\nFirst 20 cases (sorted by similarity ascending):")
    cols_to_show = ['id1', 'id2', 'similarity_score', 'full_name_L', 'full_name_R', 
                    'season_year_L', 'season_year_R', 'birth_year_L', 'birth_year_R', 'label']
    display_cols = [c for c in cols_to_show if c in lr_low_true.columns]
    
    lr_low_true_sorted = lr_low_true.sort_values('similarity_score', ascending=True)
    display(lr_low_true_sorted[display_cols].head(20))
    
    # Analyze name differences for lowest similarity cases
    print("\nTop 10 lowest similarity cases (detailed name analysis):")
    top10_lowest = lr_low_true_sorted.head(10)
    for idx, row in top10_lowest.iterrows():
        name_l = str(row.get('full_name_L', '')).lower().strip()
        name_r = str(row.get('full_name_R', '')).lower().strip()
        print(f"\n  Case (sim={row['similarity_score']:.3f}):")
        print(f"    Name L: '{name_l}'")
        print(f"    Name R: '{name_r}'")
        print(f"    Length L: {len(name_l)}, Length R: {len(name_r)}")
        print(f"    Season: {row.get('season_year_L', 'N/A')} vs {row.get('season_year_R', 'N/A')}")
        print(f"    Birth: {row.get('birth_year_L', 'N/A')} vs {row.get('birth_year_R', 'N/A')}")
        
        # Check for encoding issues
        if '\\x' in name_l or '\\x' in name_r:
            print(f"    WARNING: Potential encoding issue detected!")
        
        # Check for suffix differences (jr, iii, etc.)
        suffixes = ['jr', 'ii', 'iii', 'iv', 'sr']
        has_suffix_l = any(name_l.endswith(f' {s}') or name_l.endswith(f' {s}.') for s in suffixes)
        has_suffix_r = any(name_r.endswith(f' {s}') or name_r.endswith(f' {s}.') for s in suffixes)
        if has_suffix_l != has_suffix_r:
            print(f"    NOTE: Suffix difference detected (L has suffix: {has_suffix_l}, R has suffix: {has_suffix_r})")

print("\n" + "="*80)
print("Done.")
print("="*80)


1. LS: High-similarity FALSE cases (similarity >= 0.99)
Found 1 cases

Full details:


,id1,id2,similarity_score,full_name_L,full_name_S,season_year_L,season_year_S,birth_year_L,birth_year_S,label
488,4329342015|2015|L,4557592015|2015|S,1.0,chris young,chris young,2015,2015,1979,1984,False



Core ID comparison:
  Row 488: core1='4329342015|2015' vs core2='4557592015|2015' (match: False)
    Name L: 'chris young'
    Name S: 'chris young'
    Season: 2015 vs 2015
    Birth: 1979 vs 1984


2. LR: Low-similarity TRUE cases (similarity < 0.60)
Found 5 cases

Summary statistics:
  Similarity range: [0.550, 0.550]
  Mean similarity: 0.550
  Median similarity: 0.550

  Birth year differences:
    Same year: 0 cases
    Diff by 1: 5 cases
    Diff by 2+: 0 cases
    Missing: 0 cases

  Season year matches: 5 / 5

First 20 cases (sorted by similarity ascending):


,id1,id2,similarity_score,full_name_L,full_name_R,season_year_L,season_year_R,birth_year_L,birth_year_R,label
338,4770032011|2011|L,4770032011|2011|R,0.55,jonathon niese,jon niese,2011,2011,1986,1987,True
339,4770032014|2014|L,4770032014|2014|R,0.55,jonathon niese,jon niese,2014,2014,1986,1987,True
345,4770032008|2008|L,4770032008|2008|R,0.55,jonathon niese,jon niese,2008,2008,1986,1987,True
448,4770032010|2010|L,4770032010|2010|R,0.55,jonathon niese,jon niese,2010,2010,1986,1987,True
497,4770032009|2009|L,4770032009|2009|R,0.55,jonathon niese,jon niese,2009,2009,1986,1987,True



Top 10 lowest similarity cases (detailed name analysis):

  Case (sim=0.550):
    Name L: 'jonathon niese'
    Name R: 'jon niese'
    Length L: 14, Length R: 9
    Season: 2011 vs 2011
    Birth: 1986 vs 1987

  Case (sim=0.550):
    Name L: 'jonathon niese'
    Name R: 'jon niese'
    Length L: 14, Length R: 9
    Season: 2014 vs 2014
    Birth: 1986 vs 1987

  Case (sim=0.550):
    Name L: 'jonathon niese'
    Name R: 'jon niese'
    Length L: 14, Length R: 9
    Season: 2008 vs 2008
    Birth: 1986 vs 1987

  Case (sim=0.550):
    Name L: 'jonathon niese'
    Name R: 'jon niese'
    Length L: 14, Length R: 9
    Season: 2010 vs 2010
    Birth: 1986 vs 1987

  Case (sim=0.550):
    Name L: 'jonathon niese'
    Name R: 'jon niese'
    Length L: 14, Length R: 9
    Season: 2009 vs 2009
    Birth: 1986 vs 1987

Done.


## 8. Split ground truth into train/validation/test sets
This ensures no data leakage: all samples from the same player_id stay in the same split

In [83]:
SAMPLE_OUT_DIR = OUTPUT_DIR / 'gt' / 'samples'
GT_SPLIT_DIR = OUTPUT_DIR / 'gt' / 'splits'
GT_SPLIT_DIR.mkdir(parents=True, exist_ok=True)

# Load ground truth files using _load_gt function (defined in Section 7)
# Note: Ensure Cell 54 (Section 7) has been executed to define _load_gt function
df_lr = _load_gt('LR')
df_ls = _load_gt('LS')


def extract_player_id_from_id(id_str: str) -> str:
    """Extract player_id from id string (format: player_id|season_year|L/R/S)."""
    # Remove trailing |L/|R/|S
    core = str(id_str).replace('|L', '').replace('|R', '').replace('|S', '')
    # Extract player_id (first part before |)
    parts = core.split('|')
    if len(parts) >= 1:
        return parts[0]  # player_id
    return core


def split_gt_by_player_id(df: pd.DataFrame, edge: str, 
                          train_ratio: float = 0.6,
                          val_ratio: float = 0.2,
                          test_ratio: float = 0.2,
                          seed: int = 42) -> dict:
    """
    Split ground truth by player_id groups to avoid data leakage.
    
    Strategy:
    1. Extract player_id from id1 (for TRUE cases, id1 and id2 have same player_id)
    2. Group samples by player_id
    3. Split player_id groups (not individual samples) into train/val/test
    4. Ensure label distribution is preserved across splits
    
    Returns dict with keys: 'train', 'val', 'test', each containing a DataFrame
    """
    df = df.copy()
    
    # Normalize labels
    df['label_norm'] = df['label'].astype('string').fillna('').str.strip().str.upper()
    
    # Extract player_id from id1
    # For TRUE cases: id1 and id2 have same player_id, so either works
    # For FALSE cases: we use id1 as the primary entity identifier
    df['player_id'] = df['id1'].apply(extract_player_id_from_id)
    
    # Group by player_id and collect indices
    player_groups = df.groupby('player_id').apply(lambda x: x.index.tolist()).to_dict()
    player_ids = list(player_groups.keys())
    
    print(f"\n{edge}: Found {len(player_ids)} unique player_ids from {len(df)} samples")
    
    # Calculate label distribution for stratification
    # For each player_id group, determine its primary label (most common label in that group)
    player_labels = {}
    for pid, indices in player_groups.items():
        labels = df.loc[indices, 'label_norm'].value_counts()
        player_labels[pid] = labels.index[0] if len(labels) > 0 else 'FALSE'
    
    # Create player_id-level DataFrame for splitting
    player_df = pd.DataFrame({
        'player_id': player_ids,
        'primary_label': [player_labels[pid] for pid in player_ids],
        'sample_count': [len(player_groups[pid]) for pid in player_ids]
    })
    
    # First split: train vs (val+test)
    train_players, temp_players = train_test_split(
        player_df,
        test_size=(val_ratio + test_ratio),
        stratify=player_df['primary_label'],
        random_state=seed
    )
    
    # Second split: val vs test
    val_size = val_ratio / (val_ratio + test_ratio)
    val_players, test_players = train_test_split(
        temp_players,
        test_size=(1 - val_size),
        stratify=temp_players['primary_label'],
        random_state=seed
    )
    
    # Collect samples for each split
    train_indices = []
    for pid in train_players['player_id']:
        train_indices.extend(player_groups[pid])
    
    val_indices = []
    for pid in val_players['player_id']:
        val_indices.extend(player_groups[pid])
    
    test_indices = []
    for pid in test_players['player_id']:
        test_indices.extend(player_groups[pid])
    
    # Create split DataFrames
    splits = {
        'train': df.loc[train_indices].drop(columns=['player_id', 'label_norm']).reset_index(drop=True),
        'val': df.loc[val_indices].drop(columns=['player_id', 'label_norm']).reset_index(drop=True),
        'test': df.loc[test_indices].drop(columns=['player_id', 'label_norm']).reset_index(drop=True)
    }
    
    # Print statistics
    print(f"\n{edge} Split Statistics:")
    for split_name, split_df in splits.items():
        n = len(split_df)
        lab = split_df['label'].astype('string').fillna('').str.strip().str.upper()
        n_true = (lab == 'TRUE').sum()
        n_false = (lab == 'FALSE').sum()
        print(f"  {split_name:5s}: {n:4d} samples ({n/len(df)*100:5.1f}%) | "
              f"TRUE: {n_true:3d} ({n_true/max(n,1)*100:5.1f}%) | "
              f"FALSE: {n_false:3d} ({n_false/max(n,1)*100:5.1f}%)")
    
    # Verify no player_id leakage
    train_pids = set(train_players['player_id'])
    val_pids = set(val_players['player_id'])
    test_pids = set(test_players['player_id'])
    
    overlap_train_val = train_pids & val_pids
    overlap_train_test = train_pids & test_pids
    overlap_val_test = val_pids & test_pids
    
    print(f"\n{edge} Leakage Check:")
    print(f"  Train-Val overlap: {len(overlap_train_val)} player_ids")
    print(f"  Train-Test overlap: {len(overlap_train_test)} player_ids")
    print(f"  Val-Test overlap: {len(overlap_val_test)} player_ids")
    
    if len(overlap_train_val) == 0 and len(overlap_train_test) == 0 and len(overlap_val_test) == 0:
        print(f"  ✓ No player_id leakage detected!")
    else:
        print(f"  ⚠ WARNING: Player_id leakage detected!")
    
    return splits


# Load ground truth files using _load_gt function (already loaded above, but keeping for clarity)
# Note: The files are already loaded at the beginning of this cell using _load_gt
# This section is kept for backward compatibility, but the actual loading happens above

# Split both edges
print("="*80)
print("Splitting Ground Truth by Player ID Groups (No Data Leakage)")
print("="*80)

splits_lr = split_gt_by_player_id(df_lr, 'LR', train_ratio=0.6, val_ratio=0.2, test_ratio=0.2, seed=42)
splits_ls = split_gt_by_player_id(df_ls, 'LS', train_ratio=0.6, val_ratio=0.2, test_ratio=0.2, seed=42)

# Save splits
print("\n" + "="*80)
print("Saving split files...")
print("="*80)

for edge, splits in [('LR', splits_lr), ('LS', splits_ls)]:
    for split_name, split_df in splits.items():
        out_path = GT_SPLIT_DIR / f'gt_{edge}_{split_name}.csv'
        split_df.to_csv(out_path, index=False)
        print(f"Saved: {out_path} ({len(split_df)} rows)")

print("\n" + "="*80)
print("Split complete. All files saved to:", GT_SPLIT_DIR)
print("="*80)


Splitting Ground Truth by Player ID Groups (No Data Leakage)

LR: Found 451 unique player_ids from 500 samples

LR Split Statistics:
  train:  304 samples ( 60.8%) | TRUE:  90 ( 29.6%) | FALSE: 214 ( 70.4%)
  val  :   96 samples ( 19.2%) | TRUE:  30 ( 31.2%) | FALSE:  66 ( 68.8%)
  test :  100 samples ( 20.0%) | TRUE:  30 ( 30.0%) | FALSE:  70 ( 70.0%)

LR Leakage Check:
  Train-Val overlap: 0 player_ids
  Train-Test overlap: 0 player_ids
  Val-Test overlap: 0 player_ids
  ✓ No player_id leakage detected!

LS: Found 444 unique player_ids from 500 samples


/var/folders/fh/_tq_jbr50895yx1hkbgpct5m0000gn/T/ipykernel_2714/3673439390.py:49: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  player_groups = df.groupby('player_id').apply(lambda x: x.index.tolist()).to_dict()
/var/folders/fh/_tq_jbr50895yx1hkbgpct5m0000gn/T/ipykernel_2714/3673439390.py:49: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  player_groups = df.groupby('player_id').apply(lambda x: x.index.tolist


LS Split Statistics:
  train:  296 samples ( 59.2%) | TRUE: 142 ( 48.0%) | FALSE: 154 ( 52.0%)
  val  :  101 samples ( 20.2%) | TRUE:  49 ( 48.5%) | FALSE:  52 ( 51.5%)
  test :  103 samples ( 20.6%) | TRUE:  47 ( 45.6%) | FALSE:  56 ( 54.4%)

LS Leakage Check:
  Train-Val overlap: 0 player_ids
  Train-Test overlap: 0 player_ids
  Val-Test overlap: 0 player_ids
  ✓ No player_id leakage detected!

Saving split files...
Saved: /Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/gt/splits/gt_LR_train.csv (304 rows)
Saved: /Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/gt/splits/gt_LR_val.csv (96 rows)
Saved: /Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/gt/splits/gt_LR_test.csv (100 rows)
Saved: /Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/gt/splits/gt_LS_train.csv (296 rows)
Saved: /Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/gt/splits/gt_LS_val.csv (101 rows